# BDA2 - Spark SQL

Redo the exercises BDA1 using Spark SQL whenever possible. There are two ways to write queries in SparkSQL - using built-in API functions or running SQL-like
queries. To pass this lab, you need to use built-in API functions for all the questions, in addition, SQL-like queries for the second question. The slides of this link (https://www.ida.liu.se/~732A54/lab/SparkSQLQuickIntro.pdf) show some examples of Spark SQL.

## Setup

The following code is used to prepare the data files, execute only once and **be careful!**

In [1]:
import pyspark
import pyspark.sql
from pyspark.sql import SQLContext, Row
from pyspark.sql import functions as F
from pyspark.sql import Window
import csv
import math
import sys
import pandas as pd

tr_path = "../data/temperature-readings-tiny.csv"
tr_big_path = "../data/temperature-readings-big.csv"
pr_path = "../data/precipitation-readings-tiny.csv"
sro_path = "../data/stations-Ostergotland.csv"
sr_path = "../data/stations.csv"

In [2]:
sc = pyspark.SparkContext(appName = "Temperature")
sc_sql = pyspark.SQLContext(sc)
temperature_readings = sc.textFile(tr_path)
temperature_readings_big = sc.textFile(tr_big_path)
precipitation_readings = sc.textFile(pr_path)
station_oster_readings = sc.textFile(sro_path)
station_readings = sc.textFile(sr_path)

In [ ]:
#res = temperature_readings.map(lambda l: l.split(";"))
#res = res.map(lambda p: Row(station=int(p[0]), date=p[1], year=int(p[1].split("-")[0]), time=p[2], temp=float(p[3]), quality=p[4]))
#res = sc_sql.createDataFrame(res)
#res.head(5)

## Assignment 1

What are the lowest and highest temperatures measured each year for the period 1950- 2014. Provide the lists sorted in the descending order with respect to the maximum temperature. In this exercise you will use the `temperature-readings.csv` file.

In [3]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014)).groupBy('year')
res_max = res.agg(F.max('temp').alias('temp_max'))
res_min = res.agg(F.min('temp').alias('temp_min'))

global df

df = res_max.join(res_min, "year").orderBy('year', ascending=False).toPandas()

CPU times: user 58 ms, sys: 20.8 ms, total: 78.8 ms
Wall time: 5.76 s


In [4]:
df.head(10)

,year,temp_max,temp_min
0,2014,29.1,-24.3
1,2013,10.2,-13.3
2,2012,5.6,0.1
3,2011,5.3,-1.2
4,1961,19.0,-23.5
5,1960,29.0,-28.3
6,1959,28.2,-23.2
7,1958,28.1,-27.9
8,1957,25.2,-19.9
9,1956,26.0,-30.0


**Task 1a:** What are the lowest and highest temperature measured each year for the period 1950 to 2014? Provide the list sorted in the descending order with respect to the maximum temperature. Extend the program to include the station number (not the station name) where the maximum/minimum temperature was measured.

In [13]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), year=int(p[1].split("-")[0]), temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))

w = Window.partitionBy('year')

res_max = res.withColumn('temp_max', F.max('temp').over(w)).where(F.col('temp') == F.col('temp_max')).drop('temp')
res_min = res.withColumn('temp_min', F.min('temp').over(w)).where(F.col('temp') == F.col('temp_min')).drop('temp')

global df

df = res_max.join(res_min, "year").orderBy('year', ascending=False).toPandas()

CPU times: user 68.1 ms, sys: 1.07 ms, total: 69.2 ms
Wall time: 1.33 s


In [14]:
df.head(10)

,year,station,temp_max,station,temp_min
0,2014,102170,29.1,102170,-24.3
1,2013,102170,10.2,102170,-13.3
2,2012,102170,5.6,102170,0.1
3,2011,102170,5.3,102170,-1.2
4,1961,102190,19.0,102190,-23.5
5,1960,102190,29.0,102190,-28.3
6,1959,102190,28.2,102190,-23.2
7,1958,102190,28.1,102190,-27.9
8,1957,102190,25.2,102190,-19.9
9,1956,102190,26.0,102190,-30.0


## Assignment 2

**Task 2:** Count the number of readings for each month in the period of 1950-2014 which are higher than 10 degrees. Repeat the exercise, this time taking only distinct readings from each station. That is, if a station reported a reading above 10 degrees in some month, then it appears only once in the count for that month. In this exercise you will use the `temperature-readings.csv` file.

The output should contain the following information:

Year, month, count

In [79]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), date=p[1], year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), yearMonth=p[1][0:7], time=p[2], temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))
res = res.filter(res['temp'] > 10.0)

res = res.groupBy('yearMonth').count().select('yearMonth', F.col('count'))
res = res.select(res['yearMonth'][0:4].cast("Int").alias('year'), res['yearMonth'][6:8].cast("Int").alias('month'), res['count']).orderBy(['year', 'month'], ascending=False)

global df
df = res.toPandas()

CPU times: user 47 ms, sys: 14 ms, total: 60.9 ms
Wall time: 882 ms


In [80]:
res.head(10)

,year,month,count
0,2014,11,1
1,2014,10,17
2,2014,9,25
3,2014,8,56
4,2014,7,62
5,2014,6,54
6,2014,5,31
7,2014,4,9
8,2013,11,1
9,1961,5,39


Now we repeat the exercise where we take only distinct readings from each station.

In [89]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), date=p[1], year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), yearMonth=p[1][0:7], time=p[2], temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1950) & (res['year'] <= 2014))
res = res.filter(res['temp'] > 10.0)

res = res.groupBy('yearMonth').count().select('yearMonth', F.col('count'))
res = res.groupBy('yearMonth').count().select('yearMonth', F.col('count'))
res = res.select(res['yearMonth'][0:4].cast("Int").alias('year'), res['yearMonth'][6:8].cast("Int").alias('month'), res['count']).orderBy(['year', 'month'], ascending=False)

global df
df = res.toPandas()

In [90]:
df.head(10)

,year,month,count
0,2014,11,1
1,2014,10,1
2,2014,9,1
3,2014,8,1
4,2014,7,1
5,2014,6,1
6,2014,5,1
7,2014,4,1
8,2013,11,1
9,1961,5,1


In [17]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda l: (int(l[0]), int(l[1][0:4]), int(l[1][5:7]), float(l[3])))
res = res.filter(lambda m: int(m[1]) >= 1950 and int(m[1]) <= 2014 and m[3] > 10.0)
res = res.map(lambda m: (str(m[0]) + "-" + str(m[1]) + "-" + str(m[2]).zfill(2), 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.map(lambda m: (m[0][-7:], 1))
res = res.reduceByKey(lambda x, y: x + y)
res = res.sortBy(lambda m: m[0], ascending=False)
res = res.map(lambda m: (m[0][0:4], m[0][5:7], m[1]))

global df
df = pd.DataFrame(res.collect(), columns = ['year', 'month', 'count'])

CPU times: user 47 ms, sys: 5.55 ms, total: 52.6 ms
Wall time: 473 ms


In [18]:
df.head(10)

,year,month,count
0,2014,11,1
1,2014,10,1
2,2014,09,1
3,2014,08,1
4,2014,07,1
5,2014,06,1
6,2014,05,1
7,2014,04,1
8,2013,11,1
9,1961,05,1


## Assignment 3

Find the average monthly temperature for each available station in Sweden. Your result should include average temperature for each station for each month in the period of 1960-2014. Bear in mind that not every station has the readings for each month in this timeframe. In this exercise you will use the `temperature-readings.csvfile`.

The output should contain the following information:

Year, month, station number, average monthly temperature

In [191]:
res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda p: Row(station=int(p[0]), date=p[1], year=int(p[1].split("-")[0]), month=int(p[1].split("-")[1]), yearMonthStation=p[1][0:7] + '-' + p[0], temp=float(p[3])))
res = sc_sql.createDataFrame(res)
res = res.filter((res['year'] >= 1960) & (res['year'] <= 2014)).groupBy('yearMonthStation')

res = res.agg(F.avg('temp'))
res = res.select(res['yearMonthStation'][0:4].cast('Int').alias('year'),
                 res['yearMonthStation'][6:2].cast('Int').alias('month'),
                 res['yearMonthStation'][9:20].cast('Int').alias('station'),
                 F.round(res['avg(temp)'], 2).alias('temp_avg'))

global df
df = res.orderBy(['year', 'month', 'station'], ascending=False).toPandas()

In [192]:
df.head(10)

,year,month,station,temp_avg
0,2014,12,102170,-5.80
1,2014,11,102170,2.45
2,2014,10,102170,7.11
3,2014,9,102170,8.58
4,2014,8,102170,13.87
5,2014,7,102170,19.66
6,2014,6,102170,14.44
7,2014,5,102170,10.76
8,2014,4,102170,4.78
9,2014,3,102170,1.90


In [182]:
%%time

res = temperature_readings.map(lambda l: l.split(";"))
res = res.map(lambda m: (m[0] + "-" + m[1][0:7], int(m[1][0:4]), float(m[3])))
res = res.filter(lambda m: m[1] >= 1960 and m[1] <= 2014)
res = res.map(lambda m: (m[0], (m[2], 1)))
res = res.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
res = res.map(lambda m: (int(m[0][:-8]), int(m[0][-7:-3]), str(m[0][-2:]).zfill(2), round(m[1][0]/m[1][1], 2)))
res = res.sortBy(lambda m: (m[1], m[2], m[0]), ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['station', 'year', 'month', 'average-temperature'])

CPU times: user 48 ms, sys: 4.47 ms, total: 52.5 ms
Wall time: 387 ms


In [183]:
df.head(20)

,station,year,month,average-temperature
0,102170,2014,12,-5.80
1,102170,2014,11,2.45
2,102170,2014,10,7.11
3,102170,2014,09,8.58
4,102170,2014,08,13.87
5,102170,2014,07,19.66
6,102170,2014,06,14.44
7,102170,2014,05,10.76
8,102170,2014,04,4.78
9,102170,2014,03,1.90


## Assignment 4

Provide  a  list  of  stations  with  their  associated  maximum  measured  temperatures  and maximum  measured  daily  precipitation.  Show  only  those  stations  where  the  maximum temperature is between 25 and 30 degrees and maximum daily precipitation is between 100 mm and 200mm. In this exercise you will use the `temperature-readings.csv` and `precipitation-readings.csv` files. 

The output should contain the following information:

Station number, maximum measured temperature, maximum daily precipitation

In [ ]:
%%time

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_t = temperature_readings.map(lambda l: l.split(";"))

res_p = res_p.map(lambda m: (int(m[0]), float(m[3])))
res_p = res_p.filter(lambda m: m[1] > 100 and m[1] < 200)
res_p = res_p.reduceByKey(lambda x, y: max(x, y))

res_t = res_t.map(lambda m: (int(m[0]), float(m[3])))
res_t = res_t.filter(lambda m: m[1] > 20 and m[1] < 30)
res_t = res_t.reduceByKey(lambda x, y: max(x, y))

res = res_t.join(res_p)
res = res.map(lambda m: (m[0], m[1][0], m[1][1]))

global df
df = pd.DataFrame(res.collect(), columns=['station', 'max_temperature', 'max_precipitation'])

In [ ]:
df.head()

## Assignment 5

Calculate the average monthly precipitation for the Östergotland region (list of stations is provided in the separate file) for the period 1993-2016. In order to do this, you will first need to calculate  the  total  monthly  precipitation  for  each  station  before calculating the monthly average (by averaging over stations). In this exercise you will use the `precipitation-readings.csv` and `stations-Ostergotland.csv` files.

HINT (not for the SparkSQL lab): Avoid using joins here! `stations-Ostergotland.csv` is small  and if distributed will cause a number of unnecessary shuffles when joined with precipitation RDD. If you distribute `precipitation-readings.csv` then either repartition your stations RDD to 1 partition or make use of the collect to acquire a python list and broadcast function to broadcast the list to all nodes.

The output should contain the following information:

Year, month, average monthly precipitation

In [ ]:
%%time

# Don't remove those, it will break as we're using %%time
global res_p
global res_s

res_p = precipitation_readings.map(lambda l: l.split(";"))
res_s = station_oster_readings.map(lambda l: l.split(";"))

res_s = res_s.map(lambda m: int(m[0])).collect()
res_s = sc.broadcast(value=res_s)

res_p = res_p.map(lambda m: (int(m[0]), m[1], float(m[3])))
res_p = res_p.filter(lambda m: int(m[1][0:4]) >= 1993 and int(m[1][0:4]) <= 2016)

res = res_p.filter(lambda m: m[0] in res_s.value)
# This changes the amount of entries as it only takes year-month, not year-month-date, thus the resulting number of measurementes will me lower. Which one is correct?
#res = res.map(lambda m: (str(m[0]) + "-" + m[1][0:7], m[2]))
res = res.map(lambda m: (str(m[0]) + "-" + m[1][0:10], m[2]))
res = res.reduceByKey(lambda x, y: x + y)

res = res.map(lambda m: (m[0][6:10] + "-" + m[0][11:13], (m[1], 1)))
res = res.reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))
res = res.map(lambda m: (int(m[0][0:4]), m[0][5:8], m[1][0]/m[1][1]))
res = res.sortBy(lambda m: (m[0], m[1]), ascending=False)

global df
df = pd.DataFrame(res.collect(), columns=['year', 'month', 'average_precipitation'])

In [ ]:
df.head(10)

## Assignment 6

Compare the average monthly temperature (find the difference) in the period 1950-2014 for all stations in Östergotland with long-term monthly averages in the period of 1950-1980. Make a plot of your results. 

HINT: The first step is to find the monthly averages for each station. Then, you can average over all stations to acquire the average temperature for a specific year and month. This RDD/DataFrame can beu sed to compute the long-term average by averaging over all the years in the interval.

The out put should contain the following information:

Year, month, difference

**Answer:** It seems like this exercise has been removed from last year to this year, as the instructions have chnaged from 1. april to 2. april 2019.

In [ ]:
sc.stop()